In [131]:
import pandas as pd
from linearmodels import IVGMM, OLS
from linearmodels.iv.results import compare


# Read Excel file into a DataFrame
data = pd.read_excel('data/base_dados_modelo.xlsx')

colunas_modelos = ["date", "selic", "JUROS_NATURAL", "COMMODITIES",
                   "IBC_BR", "gap_inflacao", "GAP_EXP_INF", "EXP_PIB",
                   'selic_lag1', 'selic_lag2', 'selic_lag3', 'selic_lag4',
                   'IBC_BR_lag1', 'IBC_BR_lag2', 'IBC_BR_lag3', 'IBC_BR_lag4',
                   'COMMODITIES_lag1', 'COMMODITIES_lag2', 'COMMODITIES_lag3',
                   'COMMODITIES_lag4', 'gap_inflacao_lag1', 'gap_inflacao_lag2',
                   'gap_inflacao_lag3', 'gap_inflacao_lag4', 'LOSS_12',
                   'LOSS_24', 'LOSS_36', 'LOSS_48', "LOSS_60"]

data_model = data[colunas_modelos]

# Drop rows with missing values
data_model = data_model.dropna()

# MODELO 1
m1_lm_formula = 'selic ~ JUROS_NATURAL + gap_inflacao + LOSS_12 - 1'
m1_lm = OLS.from_formula(m1_lm_formula, data=data_model).fit()

m1_gmm_formula = 'selic ~ [JUROS_NATURAL + gap_inflacao + LOSS_12 - 1 ~ selic_lag1 + selic_lag2 + selic_lag3 + IBC_BR_lag1 + IBC_BR_lag2 + IBC_BR_lag3 + COMMODITIES_lag1 + COMMODITIES_lag2 + COMMODITIES_lag3]'
m1_gmm = IVGMM.from_formula(m1_gmm_formula, data=data_model, weight_type='robust').fit()

# MODELO 2
m2_lm_formula = 'selic ~ selic_lag1 + JUROS_NATURAL + GAP_EXP_INF +  EXP_PIB + LOSS_12-1'
m2_lm = OLS.from_formula(m2_lm_formula, data=data_model).fit()

m2_gmm_formula = 'selic ~ [selic_lag1 + JUROS_NATURAL + GAP_EXP_INF +  EXP_PIB + LOSS_12 -1 ~ selic_lag1+ selic_lag2+selic_lag3+selic_lag4  + IBC_BR_lag1+IBC_BR_lag2+IBC_BR_lag3+ IBC_BR_lag4 +COMMODITIES_lag1+ COMMODITIES_lag2 +COMMODITIES_lag3+ COMMODITIES_lag4+gap_inflacao_lag1+gap_inflacao_lag2+gap_inflacao_lag3+gap_inflacao_lag4-1]'
m2_gmm = IVGMM.from_formula(m2_gmm_formula, data=data_model, weight_type='robust').fit()

# MODELO 3

data_model['NEG_CRED'] = (data_model['LOSS_12'] < 0).astype(int)
data_model['POS_CRED'] = (data_model['LOSS_12'] > 0).astype(int)

m3_lm_formula = 'selic ~ selic_lag1 + JUROS_NATURAL + GAP_EXP_INF + EXP_PIB + LOSS_12:POS_CRED + NEG_CRED:LOSS_12 - 1'
m3_lm = OLS.from_formula(m3_lm_formula, data=data_model).fit()

m3_gmm_formula = 'selic ~ [selic_lag1 + JUROS_NATURAL + GAP_EXP_INF + EXP_PIB + LOSS_12:POS_CRED + NEG_CRED:LOSS_12 - 1 ~ selic_lag1+ selic_lag2+selic_lag3+selic_lag4  + IBC_BR_lag1+IBC_BR_lag2+IBC_BR_lag3+ IBC_BR_lag4 +COMMODITIES_lag1+ COMMODITIES_lag2 +COMMODITIES_lag3+ COMMODITIES_lag4+gap_inflacao_lag1+gap_inflacao_lag2+gap_inflacao_lag3+gap_inflacao_lag4]'
m3_gmm = IVGMM.from_formula(m3_gmm_formula, data=data_model, weight_type='robust').fit()



In [132]:
m1_comparison_table = compare({'OLS': m1_lm, 'IV-GMM': m1_gmm}, stars=True)
m2_comparison_table = compare({'OLS': m2_lm, 'IV-GMM': m2_gmm}, stars=True)
m3_comparison_table = compare({'OLS': m3_lm, 'IV-GMM': m3_gmm}, stars=True)


# Modelo teórico derivado

Esta tabela mostra os resultados para o modelo teórico derivado, observa-se que para os dois métodos de estimação o coeficiente associado a perda de credibilidade é positivo e estatisticamente significante. Além disso, pode-se observar que o gap de inflação também tem as mesmas características. Esses resultados comprovam a ideia de que o BC aumenta a taxa de juros quando a inflação está acima da meta e, como o modelo teórico demonstra, a autoridade monetária busca compensar perdas de credibilidade com taxas de juros maiores.


In [128]:
print(m1_comparison_table)

                   Model Comparison                  
                               OLS             IV-GMM
-----------------------------------------------------
Dep. Variable                selic              selic
Estimator                      OLS             IV-GMM
No. Observations               186                186
Cov. Est.                   robust             robust
R-squared                   0.9835             0.9731
Adj. R-squared              0.9832             0.9726
F-statistic                 9745.3             5470.3
P-value (F-stat)            0.0000             0.0000
==================     ===========        ===========
JUROS_NATURAL            2.1348***          2.0094***
                          (83.127)           (34.676)
gap_inflacao             0.4998***          0.5302***
                          (5.5111)           (4.2301)
LOSS_12                  0.1390***          0.3050***
                          (5.0970)           (4.3303)
==================== =======

# Modelo forward looking com suavização

Esta tabela mostra os resultados para o modelo forward looking com suavização. A variável de gap de inflação esperada é estatisticamente significante para os dois métodos, indicando que o BC de fato segue uma regra forward looking. O componente de suavização apresenta um valor elevado em ambas as estimativas, indicando um alto grau de suavização da Selic. O coeficiente associado a variável Perda de Credibilidade se mostra significante, porém com valor negativo, algo que não era esperado. Uma possível explicação para isso é que uma memória de 12 meses é muita pequena para ter algum efeito nas decisões do BC


In [129]:
print(m2_comparison_table)

                    Model Comparison                   
                                OLS              IV-GMM
-------------------------------------------------------
Dep. Variable                 selic               selic
Estimator                       OLS              IV-GMM
No. Observations                186                 186
Cov. Est.                    robust              robust
R-squared                    0.9995              0.9971
Adj. R-squared               0.9995              0.9970
F-statistic               3.854e+05           8.729e+04
P-value (F-stat)             0.0000              0.0000
==================     ============        ============
selic_lag1                0.9688***           1.1848***
                           (77.927)            (14.054)
JUROS_NATURAL               0.0675*           -0.7814**
                           (1.7852)           (-2.4883)
GAP_EXP_INF               0.2098***           0.7008***
                           (8.8685)            (

# Modelo forward looking com assimetria

Esta tabela mostra os resultados do modelo com assimetria na reação do BC, novamente verificamos que o componente de suavização é significativo nos dois modelos. Assim como o coeficiente associado a variável de expectativa da inflação que caracteriza o comportamento forward looking do BC.
O coeficiente referente a variável de expectativa de inflação, é estatisticamente significante e positivo, indicando que uma regra forward looking parece ser uma boa representação de como o BC toma suas decisões. Por outro lado, apesar de estatisticamente significantes, os coeficientes associados a perdas de credibilidade positiva (inflação acima da meta) são pequenos e negativos, isso pode ser um indicativo que a memória de 12 meses não é a ideal para capturar a realidade da política monetária.

In [133]:
print(m3_comparison_table)

                    Model Comparison                   
                                OLS              IV-GMM
-------------------------------------------------------
Dep. Variable                 selic               selic
Estimator                       OLS              IV-GMM
No. Observations                186                 186
Cov. Est.                    robust              robust
R-squared                    0.9995              0.9954
Adj. R-squared               0.9995              0.9952
F-statistic               4.547e+05           6.503e+04
P-value (F-stat)             0.0000              0.0000
==================     ============        ============
selic_lag1                0.9723***           0.9149***
                           (76.981)            (5.8348)
JUROS_NATURAL                0.0536              0.4489
                           (1.3617)            (0.6569)
GAP_EXP_INF               0.2195***              0.3586
                           (8.7101)            (